![alt text](DataKind_orange.png)

# Omidyar Extractives Project 1
## Clean Contract Text (Notebook 4 of 8)
### 1. Reads in contract text
### 2. Strips HTML from text
### 3. Cleans text for NLP prep (remove unicode, special characters, stopwords, etc.)
### 4. Outputs cleaned text appended to dataframe from Notebook 1

In [1]:
import re
import string
import pandas as pd
from HTMLParser import HTMLParser
import nltk
from nltk.corpus import stopwords
from langdetect import detect

import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
import matplotlib.pylab as plt
import seaborn as sns
%matplotlib inline

In [2]:
contracts = pd.read_pickle('contract_data/1_contracts_with_text.pkl')
outfile = 'contract_data/2_contracts_with_text.pkl'

In [3]:
print "Number of contracts: " + str(len(contracts))

Number of contracts: 1496


### Parse by paragraph, strip HTML, carriage returns, newline characters

In [4]:
def get_paragraphs(text):
    return re.split('\s{4,}',text)

In [5]:
class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [6]:
contracts['Text_by_Paragraph'] = contracts['Contract_Text'].apply(get_paragraphs)

In [20]:
pd.set_option('max_colwidth',200)
contracts[contracts['Text_by_Paragraph'].map(len) ==2]

,OCID,Category,Contract Name,Contract Identifier,Language,Country Name,Resource,Contract Type,Signature Date,Document Type,...,Disclosure Mode,Retrieval Date,Pdf Url,Deal Number,Contract Note,Matrix Page,Annotation Category,Annotation Text,Contract_Text,Text_by_Paragraph
215,ocds-591adf-7104921871,rc,"Kosmos Energy Offshore Morocco HC, Cap Boujdour Offshore, PSA, 2011",NaN,en,Morocco,Hydrocarbons,Production or Profit Sharing Agreement,2011-07-11,Contract,...,Company,2016-08-09,https://resourcecontracts-nrgi.s3-us-west-2.amazonaws.com/2148/2148-kosmos-energy-offshore-morocco-hc-cap-boujdour-offshore-psa-2011.pdf,NaN,NaN,NaN,NaN,NaN,EX-10.27 28 a13-19720_1ex10d27.htm EX-10.27<br /><br />\nExhibit 10.27<br /><br />\nPETROLEUM AGREEMENT<br /><br />\n<br /><br />\nREGARDING<br /><br />\n<br /><br />\nTHE EXPLORATION FOR AND EXPL...,[EX-10.27 28 a13-19720_1ex10d27.htm EX-10.27<br /><br />\nExhibit 10.27<br /><br />\nPETROLEUM AGREEMENT<br /><br />\n<br /><br />\nREGARDING<br /><br />\n<br /><br />\nTHE EXPLORATION FOR AND EXP...
283,ocds-591adf-8080364797,rc,"Minera Yanacocha S.R.L., Maqui Maqui, Investment Promotion Agreement, 1994",NaN,es,Peru,Gold,Investment Promotion Agreement,1994-05-20,Contract,...,Government,2016-02-13,https://resourcecontracts-nrgi.s3-us-west-2.amazonaws.com/1684/1684-peru-minera-yanacocha-exploration-investment-contract-1994.pdf,NaN,NaN,NaN,NaN,NaN,CONTRATO DE GARANTIAS Y MEDIDAS DE PROMOCION A LA INVERSION<br /><br />\r\n<br /><br />\r\nDECRETO SUPREMO N° 0 4-94-EM<br /><br />\r\n<br /><br />\r\nSeñor Notario:<br /><br />\r\n<br /><br />\r\...,[CONTRATO DE GARANTIAS Y MEDIDAS DE PROMOCION A LA INVERSION<br /><br />\r\n<br /><br />\r\nDECRETO SUPREMO N° 0 4-94-EM<br /><br />\r\n<br /><br />\r\nSeñor Notario:<br /><br />\r\n<br /><br />\r...
303,ocds-591adf-4212712774,rc,"Compa��a Minera Oro Candente S.A., Exploration License, Investment Promotion Agreement, 2003",NaN,es,Peru,other minerals,Exploration Permit/License;Investment Promotion Agreement,2003-09-15,Contract,...,Government,2016-01-20,https://resourcecontracts-nrgi.s3-us-west-2.amazonaws.com/1609/1609-peru-compania-minera-oro-exploration-investment-contract-2003.pdf,NaN,NaN,NaN,NaN,NaN,",<br /><br />\nMINISTERIO<br /><br />\nDEENERGIA<br /><br />\ny MINAS<br /><br />\nDPDM-DGM<br /><br />\n<br /><br />\nFOLIO:<br /><br />\n<br /><br />\n211<br /><br />\nNómeros<br /><br />\n<br /...","[,<br /><br />\nMINISTERIO<br /><br />\nDEENERGIA<br /><br />\ny MINAS<br /><br />\nDPDM-DGM<br /><br />\n<br /><br />\nFOLIO:<br /><br />\n<br /><br />\n211<br /><br />\nNómeros<br /><br />\n<br ..."
342,ocds-591adf-1623268944,rc,"Eramen Minerals, Incorporated - Annex of MPSA No. 209-2005-III, 2005",ph_Sta-Cruz-Nickel-Project,en,Philippines,Nickel,Production or Profit Sharing Agreement,2005-04-19,Contract Annexe,...,Government,NaN,https://resourcecontracts-nrgi.s3-us-west-2.amazonaws.com/1098/1098-annexes-mpsa-no-209-2005-iii.pdf,NaN,NaN,NaN,NaN,NaN,︶<br /><br />\n <br /><br />\n <br /><br />\n一<br /><br />\n<br /><br />\nANNEX― A<br /><br />\n<br /><br />\nSECRETARY'S<br /><br />\nCER丁 lFiCATE<br /><br />\n<br /><br />\nν<br /><br />\n<br />...,[︶<br /><br />\n <br /><br />\n <br /><br />\n一<br /><br />\n<br /><br />\nANNEX― A<br /><br />\n<br /><br />\nSECRETARY'S<br /><br />\nCER丁 lFiCATE<br /><br />\n<br /><br />\nν<br /><br />\n<br /...
469,ocds-591adf-3068184322,rc,"Empresa Mo�ambicana de Explora��o Mineira, Minas Moatize, Limitada, Moatize, Concession, 2013",mz_dd20130413_Moatize-No-1163C_Concession_Emem_Minas,pt,Mozambique,Coal,Concession Agreement,2013-04-03,Contract,...,Government,2016-01-02,https://resourcecontracts-nrgi.s3-us-west-2.amazonaws.com/1498/1498-mozambique-emem-minas-tm-no-1163c-concession-2013.pdf,NaN,NaN,NaN,NaN,NaN,[image]<br /><br />\r\nREPÚBLICA DE MOÇAMBIQUE MINISTÉRIO DOS RECURSOS MINERAIS<br /><br />\r\n<br /><br />\r\nCONTRATO MINEIRO ENTRE<br /><br />\r\n<br /><br />\r\nO ESTADO REPRESENTADO PELO MINI...,[[image]<br /><br />\r\nREPÚBLIC

In [9]:
contracts['Text_by_Paragraph'][0]

['INVESTMENT AGREEMENT<br /><br />\n<br /><br />\nBETWEEN<br /><br />\nTHE GOVERNMENT OF MONGOLIA<br /><br />\nAND<br /><br />\nIVANHOE MINES MONGOLIA INC LLC<br /><br />\nAND<br /><br />\nIVANHOE MINES LTD<br /><br />\nAND<br /><br />\nRIO TINTO INTERNATIONAL HOLDINGS LIMITED<br /><br />\n<br /><br />\n\x0cINVESTMENT AGREEMENT<br /><br />\n<br /><br />\nIn accordance with Article 29 of the Minerals Law of Mongolia (hereinafter referred to as the<br /><br />\n\xe2\x80\x9cMinerals Law\xe2\x80\x9d), Resolution Number 40 dated 4 December 2008 and Resolution Number 57<br /><br />\ndated 16 July 2009 respectively of the State Great Khural and Government Resolution Number<br /><br />\n308 dated 6 October 2009:<br /><br />\nThe Minister of Finance, the Minister of Mineral Resources and Energy and the Minister<br /><br />\nof Nature, Environment and Tourism, together representing and upon authorization of the<br /><br />\nGovernment of Mongolia;<br /><br />\nand<br /><br />\nIvanhoe Mines Mong

In [ ]:
contracts['HTML_Stripped_Text'] = contracts['Contract_Text'].apply(strip_tags)

In [15]:
characters_to_replace = ['\xef','\xbb','\xbf','\r','\n']
for char in characters_to_replace:
    contracts['HTML_Stripped_Text'] = contracts['HTML_Stripped_Text'].str.replace(char," ")

In [16]:
contracts['TextLength'] = contracts['HTML_Stripped_Text'].str.len()

In [22]:
## Remove short documents
contracts = contracts[contracts['TextLength'] > 2000].copy()

### Clean Text

In [ ]:
unicode_re = nltk.regexp.re.compile(u'('u'\ud83c[\udf00-\udfff]|'
                        u'\ud83d[\udc00-\ude4f\ude80-\udeff]|'
                        u'[\u2600-\u26FF\u2700-\u27BF])+',
                        nltk.regexp.re.UNICODE)
url_re = r'(?:www|https?\://)\S+'

In [ ]:
def unicode_remove(text):
    try:
        return text.decode('unicode_escape').encode('ascii','ignore')
    except:
        try:
            return text.decode('string-escape').encode('ascii','ignore')
        except:
            return repr(text).translate(None,r'\\x')

def perform_lowercase(text):
    """
    Mutates and returns text where all characters are lowercased
    """
    new_text = text.lower()
    return new_text

def urlpath_replace(text):
    """
    Replaces URL with URL base path
    """
    new_text = text
    urls = nltk.regexp.re.findall(url_re,text)
    for url in urls:
        try:
            parsed = urlparse.urlparse(url)
            urlnetloc = parsed.netloc
            urlpath = parsed.path
            if urlnetloc != '':
                new_text = text.replace(url,urlnetloc)
            else:
                new_text = text.replace(url,urlpath)
            text = new_text
        except:
            pass
    return new_text

def emoji_replace_word(text):
    """
    Replaces emoji byte representations with ' EMOJI ' 
    Uses UCS-4 wide code points for the regex
    http://stackoverflow.com/questions/26568722/remove-unicode-emoji-using-re-in-python
    """
    if isinstance(text, unicode):
        new_text = nltk.regexp.re.sub(unicode_re, ' emoji ', text)
        return new_text
    else:
        return text

def remove_underscore(text):
    new_text = text.replace('_',' ')
    return new_text

def punctuation_remove(text):
    """
    Mutates and returns text where all punctuation are replaced
    """
    chars = re.escape(string.punctuation)
    return re.sub(r'['+chars+']', ' ',text)

def doublespace_remove(text):
    return re.sub(' +',' ',text)

In [ ]:
def cleaning(text):
    function_list = [
        unicode_remove, perform_lowercase, urlpath_replace, 
        emoji_replace_word, punctuation_remove, 
        remove_underscore, doublespace_remove
                ]
    for func in function_list:
        text = func(text)
    
    return text

In [ ]:
contracts['CleanText'] = contracts.apply(lambda row: row['HTML_Stripped_Text'] if row['Language'] == 'ar' else cleaning(row['HTML_Stripped_Text']),axis=1)

### Optimizing for NLP

In [ ]:
stopwords = set(stopwords.words('english'))
stopwords.update(stopword_list)
def remove_stopwords(text):
    return ' '.join([i for i in text.lower().split() if i not in stopwords])